In [24]:
import torch
torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"
# use mac gpu if available
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)
torch.set_default_device(device)

mps


In [27]:
import torch
x = torch.tensor([1.0, 2.0, 3.0]).to(device)
y = torch.tensor([4.0, 5.0, 6.0]).to(device)
print(x + y)
print(x * y)
print(torch.sin(x))
print(torch.cos(x))
x = torch.rand(5, 3)
y = torch.rand(5, 3)
print(x + y)
print(torch.matmul(x, y.T))
x

tensor([5., 7., 9.], device='mps:0')
tensor([ 4., 10., 18.], device='mps:0')
tensor([0.8415, 0.9093, 0.1411], device='mps:0')
tensor([ 0.5403, -0.4161, -0.9900], device='mps:0')
tensor([[0.4294, 1.2331, 0.4936],
        [1.6689, 1.1227, 0.1169],
        [0.0864, 1.8202, 1.6241],
        [0.8889, 0.7094, 1.3148],
        [0.3716, 0.9386, 1.1983]], device='mps:0')
tensor([[0.3892, 0.8441, 1.1768, 0.7630, 0.8565],
        [0.2939, 1.0051, 0.5271, 0.7121, 0.4392],
        [0.4096, 0.6462, 1.4831, 0.9319, 1.1240],
        [0.2493, 0.4898, 0.8116, 0.7347, 0.6815],
        [0.2151, 0.4323, 0.6893, 0.5922, 0.5647]], device='mps:0')


tensor([[0.2658, 0.9145, 0.3277],
        [0.8362, 0.4626, 0.0585],
        [0.0596, 0.8318, 0.8130],
        [0.3229, 0.2769, 0.6525],
        [0.2543, 0.2911, 0.4867]], device='mps:0')

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
inputs = tokenizer("Hello world!", return_tensors="pt") # pt means PyTorch tensor format
outputs = model(**inputs)
# print(outputs.logits)

In [8]:
print(outputs.last_hidden_state.shape) # shape: (batch_size, sequence_length, hidden_size)

torch.Size([1, 5, 768])


In [11]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1424,  0.1335, -0.1291,  ..., -0.3597, -0.0562,  0.3605],
         [-0.3506,  0.1042,  0.6244,  ..., -0.1761,  0.4834,  0.0644],
         [-0.2451, -0.1573,  0.6945,  ..., -0.5654, -0.0894, -0.1856],
         [-0.8248, -0.9119, -0.6561,  ...,  0.5074, -0.1939, -0.1659],
         [ 0.8767,  0.0352, -0.1233,  ...,  0.2720, -0.6369, -0.1585]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.9756e-01, -3.3040e-01, -7.6942e-01,  7.5799e-01,  4.6678e-01,
         -1.2035e-01,  9.1835e-01,  1.8087e-01, -7.2716e-01, -9.9991e-01,
         -4.4723e-01,  8.9104e-01,  9.6621e-01,  5.4915e-01,  9.4344e-01,
         -7.6605e-01, -6.0469e-01, -6.1654e-01,  4.0572e-01, -7.4644e-01,
          6.1739e-01,  9.9974e-01,  3.2989e-02,  2.5414e-01,  4.3106e-01,
          9.7732e-01, -8.4328e-01,  9.2297e-01,  9.4871e-01,  6.3994e-01,
         -7.3620e-01,  9.0957e-02, -9.7607e-01, -1.9115e-01, -8.1717e-01,
    

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
def summarise_text(test):
    inputs = tokenizer.encode(
        "Summarise, focus on key impacts and industries: " + text,
        truncation=True,
        padding=True,
        return_tensors="pt", # pt means PyTorch tensor format
        max_length=1024,
    )
    outputs = model.generate(
        inputs,
        num_beams=5,  # number of beams to use for beam search
        max_length=1024,  # maximum length of the generated text
        early_stopping=True,  # stop generating once an EOS token is found
        num_return_sequences=1,  # number of sequences to generate
        min_length=50,  # minimum length of the generated text
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

text = """
The AI revolution is transforming industries and society. It has led to significant advancements in fields such as machine learning, natural language processing, and robotics. The impact of AI on employment is also being explored, with some experts predicting that it could lead to a significant increase in job opportunities.
AI has the potential to revolutionize healthcare by enabling more accurate diagnoses and personalized treatment plans. It can also be used to improve transportation systems by optimizing routes and reducing traffic congestion. AI is also being explored for its potential in agriculture, where it can help farmers make more informed decisions about crop yields and pest management.
The future of AI is uncertain, but one thing is clear: it has the power to change the world.
"""
summary = summarise_text(text)
print(summary)
# print(outputs.logits)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Error while downloading from https://cdn-lfs.hf.co/t5-base/a90903540cc02cbeb7ff9f823f1a80eb778c7e22426a0e620b01c77a5ec8f5b4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1738054639&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODA1NDYzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby90NS1iYXNlL2E5MDkwMzU0MGNjMDJjYmViN2ZmOWY4MjNmMWE4MGViNzc4YzdlMjI0MjZhMGU2MjBiMDFjNzdhNWVjOGY1YjQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=oaLNb96XVKpVSeF7Ua%7EBzf0dBeAM4Cy19di9lATpv%7ESy%7Eiiq5YVC3IwmFtkRynBSKxCxOXp5MKhzU%7ELRLxAYUK8gwWJjCdLC

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.hf.co\', port=443): Max retries exceeded with url: /t5-base/a90903540cc02cbeb7ff9f823f1a80eb778c7e22426a0e620b01c77a5ec8f5b4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1738054639&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODA1NDYzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby90NS1iYXNlL2E5MDkwMzU0MGNjMDJjYmViN2ZmOWY4MjNmMWE4MGViNzc4YzdlMjI0MjZhMGU2MjBiMDFjNzdhNWVjOGY1YjQ~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=oaLNb96XVKpVSeF7Ua~Bzf0dBeAM4Cy19di9lATpv~Sy~iiq5YVC3IwmFtkRynBSKxCxOXp5MKhzU~LRLxAYUK8gwWJjCdLCNPAQ53kgitK5XIKtU7SJPJEKdXWZiWczUlVd70VMIF3C0~-fT764FhvEjcYZRLt6jGJ6NBpAOvF2sjdOg0Y2VsWRbw9Oh3UQSJcPOp9ywd2M6sEYSugl500qmJdlz1cuHUalal8Z4sefSy0aQzP8ziyKd87WVXfOo4Y3A9wNFGNQvyjeIS6EUQF6IJ-XTXzR0dvkUP~CTunQ87C0gknVKfTDVm-p4GgHNdznM7FNCWISd8IgthqoGw__&Key-Pair-Id=K3RP